# 前馈神经网络
前馈神经网络是一种最简单的神经网络，各神经元分层排列。每个神经元只与前一层的神经元相连。接收前一层的输出，并输出给下一层．各层间没有反馈。是应用最广泛、发展最迅速的人工神经网络之一。研究从20世纪60年代开始，理论研究和实际应用达到了很高的水平。

<img src="../img/feedforward.png" style="width:50%">

其分类有
1. 感知器网络
感知器（又叫感知机）是最简单的前馈网络，它主要用于模式分类，也可用在基于模式分类的学习控制和多模态控制中。感知器网络可分为单层感知器网络和多层感知器网络。
2. BP网络
BP网络是指连接权调整采用了反向传播（Back Propagation）学习算法的前馈网络。与感知器不同之处在于，BP网络的神经元变换函数采用了S形函数（Sigmoid函数），因此输出量是0~1之间的连续量，可实现从输入到输出的任意的非线性映射。
3. RBF网络
RBF网络是指隐含层神经元由RBF神经元组成的前馈网络。RBF神经元是指神经元的变换函数为RBF（Radial Basis Function，径向基函数）的神经元。典型的RBF网络由三层组成：一个输入层，一个或多个由RBF神经元组成的RBF层（隐含层），一个由线性神经元组成的输出层。


## 数据集
本节以手写数字MNIST数据集作为数据集进行训练。采用GPU训练的方法。

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# 加载数据集查看数据情况
用以设计输入和输出

In [12]:
train_dataset = torchvision.datasets.MNIST(root='../data',download=True,transform=transforms.ToTensor(),train=True)
test_dataset = torchvision.datasets.MNIST(root='../data',download=True,transform=transforms.ToTensor(),train=False)

# 查看单张图片的像素大小 1*28*28
print(train_dataset[0][0].shape)
print(test_dataset[0][0].shape)
print('训练集图片张数:',train_dataset.data.shape[0])
print('测试集图片张数:',test_dataset.data.shape[0])

torch.Size([1, 28, 28])
torch.Size([1, 28, 28])
训练集图片张数: 60000
测试集图片张数: 10000


# 输入设置
输入应该是28*28，训练集有6000张图片，那么可以将batch_size设置为100。因为时前馈神经网络，可以将隐藏层设置为100个节点

In [35]:
input_size = 28 * 28
hidden_layer = 100
nums_classes = 10
num_epoch = 200
batch_size = 100
learning_rate = 0.001

# 数据集加载

In [36]:
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=100,shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset,batch_size=100,shuffle=False)

# 模型建立

In [37]:
class FNN(nn.Module):
    def __init__(self, input_size, hidden_layer, nums_classes) -> None:
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_layer)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_layer,nums_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

device = 'cuda' if torch.cuda.is_available() else 'cpu'
fnn = FNN(input_size, hidden_layer, nums_classes).to(device)
loss_fn = nn.CrossEntropyLoss()
optmizer = torch.optim.SGD(fnn.parameters(),lr=learning_rate)


# 在训练集上训练

In [38]:
total_step = len(train_dataloader)
for epoch in range(num_epoch):
    for i,(images, labels) in enumerate(train_dataloader):
        images = images.reshape(-1,28*28)
        # 放到GPU上执行
        images = images.to(device)
        labels = labels.to(device)
        # 前向传播获得输出
        out = fnn(images)
        # 计算损失
        loss = loss_fn(out, labels)

        # 损失反向传播计算梯度
        optmizer.zero_grad() # 归零防止累加
        loss.backward() # 反向传播计算梯度
        optmizer.step() # 优化器优化梯度
        if (i+1) % 100 == 0:
            print('Epoch:[{}/{}], step:[{}/{}],loss:{:.4f}'.format(epoch+1,num_epoch,i+1,total_step,loss.item()))

Epoch:[1/200], step:[100/600],loss:2.3004
Epoch:[1/200], step:[200/600],loss:2.2914
Epoch:[1/200], step:[300/600],loss:2.2752
Epoch:[1/200], step:[400/600],loss:2.2514
Epoch:[1/200], step:[500/600],loss:2.2426
Epoch:[1/200], step:[600/600],loss:2.2376
Epoch:[2/200], step:[100/600],loss:2.2110
Epoch:[2/200], step:[200/600],loss:2.2097
Epoch:[2/200], step:[300/600],loss:2.1738
Epoch:[2/200], step:[400/600],loss:2.1582
Epoch:[2/200], step:[500/600],loss:2.1386
Epoch:[2/200], step:[600/600],loss:2.1194
Epoch:[3/200], step:[100/600],loss:2.0965
Epoch:[3/200], step:[200/600],loss:2.0691
Epoch:[3/200], step:[300/600],loss:2.0323
Epoch:[3/200], step:[400/600],loss:2.0327
Epoch:[3/200], step:[500/600],loss:1.9902
Epoch:[3/200], step:[600/600],loss:1.9518
Epoch:[4/200], step:[100/600],loss:1.9591
Epoch:[4/200], step:[200/600],loss:1.8787
Epoch:[4/200], step:[300/600],loss:1.8569
Epoch:[4/200], step:[400/600],loss:1.8988
Epoch:[4/200], step:[500/600],loss:1.8531
Epoch:[4/200], step:[600/600],loss

# 测试准确率

In [39]:
total_num = 0
correct_num = 0
with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.reshape(-1,28*28)
        # GPU上
        images = images.to(device)
        labels = labels.to(device)
        # 获得预测结果
        outputs = fnn(images)
        # max 返回每一行的最大值及其索引
        _,predicted = torch.max(outputs.data, 1)
        # print(_,predicted)

        total_num += images.size(0)
        correct_num += (labels == predicted).sum()
    print("准确率是:{}%".format(100*correct_num/total_num))


准确率是:93.37999725341797%


# 保存模型

In [ ]:
torch.save(fnn.state_dict(), 'fnn.pickle')